In [1]:
import graphlab as gl
import graphlab.aggregate as agg
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)
from numpy import log as nplog
from scipy.stats import pearsonr, spearmanr

nyt = gl.SFrame('nyt.sf')
aca = gl.SFrame('academic_articles.sf')
print nyt
print aca

[INFO] This non-commercial license of GraphLab Create is assigned to loganwls@uw.edu and will expire on October 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-94664 - Server binary: /home/loganwls/.local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1463943131.log
[INFO] GraphLab Server Version: 1.6.1


+--------+-------------------------------+-------------------------------+
| nyt_id |             byline            |          raw_headline         |
+--------+-------------------------------+-------------------------------+
|   0    | {'person': [{'organization... | {'main': "'Fat Talk' Compe... |
|   1    | {'person': [{'organization... | {'main': 'Really? The Clai... |
|   2    | {'person': [{'firstname': ... | {'main': 'The New Rules fo... |
|   3    | {'contributor': '', 'perso... | {'seo': 'Understanding Hoa... |
|   4    | {'contributor': '', 'perso... | {'main': 'New Tools to Hun... |
|   5    | {'contributor': '', 'perso... | {'seo': 'Polio Reappears i... |
|   6    | {'person': [{'organization... | {'main': 'The Roving Runne... |
|   7    | {'person': [{'organization... | {'main': 'Is It Better to ... |
|   8    | {'person': [{'organization... | {'main': 'What Not to Say ... |
|   9    | {'person': [{'organization... | {'main': 'Winning the MRSA... |
+--------+---------------

In [2]:
nyt['nyt_id'] = nyt['nyt_id'].astype(str)
# Drop any NYT articles with no citations.
edges = nyt[['nyt_id','DOI']].stack('DOI', 'DOI').dropna('DOI')
edges.rename({'nyt_id':'source', 'DOI':'target'})
edges

source,target
0,10.1016/j.bodyim.2012.05.004 ...
0,10.1016/j.bodyim.2013.02.001 ...
1,10.3945/jn.108.101360
1,10.3945/ajcn.111.015560
1,10.3945/ajcn.2010.28450d
1,10.1016/j.jada.2005.09.005 ...
1,10.1377/hlthaff.2010.1273
1,10.1080/07315724.2011.10719986 ...
1,10.3945/ajcn.2010.28450e
1,10.1017/s1368980011000656


In [3]:
edges.save('article_network/edges.csv')

In [4]:
nyt_nodes = nyt[['nyt_id', 'author', 'headline', 'news_desk', 'section_name', 'pub_date', 'topic_id']]
nyt_nodes['node_type'] = gl.SArray.from_const('nyt', nyt_nodes.num_rows())
nyt_nodes.rename({'nyt_id':'id', 'headline':'title'})
nyt_nodes

id,author,title,news_desk,section_name,pub_date,topic_id
0,JAN HOFFMAN,'Fat Talk' Compels butCarries a Cost ...,None,Health,2013-05-27 13:15:43+00:00,2
1,ANAHAD O'CONNOR,Really? The Claim: FreshProduce Has More ...,None,Health,2013-05-27 14:00:36+00:00,3
2,RONI CARYN RABIN,The New Rules forSunscreen ...,None,Health,2013-05-27 15:30:46+00:00,1
3,JAN HOFFMAN,Understanding Hoarding,National,Health,2013-05-28 00:00:00+00:00,1
4,DONALD G. McNEIL Jr,New Tools to Hunt NewViruses ...,Science,Health,2013-05-28 00:00:00+00:00,0
5,DONALD G. McNEIL Jr,Polio Reappears in theHorn of Africa ...,Science,Health,2013-05-28 00:00:00+00:00,1
6,BRIAN FIDELMAN,The Roving Runner Rides aBike ...,None,Health,2013-05-28 17:01:33+00:00,0
7,GRETCHEN REYNOLDS,Is It Better to Walk orRun? ...,None,Health,2013-05-29 00:01:50+00:00,0
8,JUDITH GRAHAM,What Not to Say at theBedside ...,None,Health,2013-05-29 13:05:04+00:00,1
9,ANAHAD O'CONNOR,Winning the MRSA Battlein Hospitals ...,None,Health,2013-05-29 17:47:59+00:00,4


In [5]:
aca_nodes = aca[['DOI', 'author', 'publication_name', 'publisher', 'title', 'pub_date', 'eigenfactor', 'open_access']]
# Convert the list of authors to a concatenated string so it is compatible with the NYT authors.
aca_nodes['author'] = aca_nodes['author'].apply(lambda x: ', '.join(x))
aca_nodes.rename({'DOI':'id'})
aca_nodes['node_type'] = gl.SArray.from_const('academic', aca_nodes.num_rows())
aca_nodes

id,author,publication_name,publisher
10.1001/jama.2013.5892,"Patty J. Nelemans, Rob A.de Bie, J. Bart Staal ...",JAMA,American MedicalAssociation (AMA) ...
10.1056/nejmsa053935,"Earl S. Ford, Julia A.Critchley, Thomas E. ...",New England Journal ofMedicine ...,New England Journal ofMedicine (NEJM/MMS) ...
10.1016/j.tics.2004.11.007 ...,"Susan E. Rogers, DanielJ. Levitin ...",Trends in CognitiveSciences ...,Elsevier BV
10.1002/oby.20384,"Debra L. Roter, KimberlyA. Gudzune, Mary ...",Obesity,Wiley-Blackwell
10.1056/NEJMp1215606,"Sean Palfrey, Judith S.Palfrey ...",New England Journal ofMedicine ...,New England Journal ofMedicine (NEJM/MMS) ...
10.1056/nejmsa042657,"Christopher P.Neukermans, Gillian D. ...",New England Journal ofMedicine ...,New England Journal ofMedicine (NEJM/MMS) ...
10.1161/01.cir.67.1.24,"T. D. Noakes, L. H. Opie,L. Higginson ...",Circulation,Ovid Technologies(Wolters Kluwer Health) ...
10.1177/0363546511433279,"J. E. Shealy, S. Kim, C.F. Ettlinger, R. J. ...",The American Journal ofSports Medicine ...,SAGE Publications
10.3357/asem.2538.2009,"Sebastien Villard, YawenYu, Thomas A. Stoffregen ...",aviat space environ med,Aerospace MedicalAssociation ...
10.1001/jama.2010.303,Brenda R. Hemmelgarn,JAMA,American MedicalAssociation (AMA) ...


In [6]:
# Add empty columns to each set of nodes so we can join them into one table
aca_cols = aca_nodes.column_names()
for c, t in zip(nyt_nodes.column_names(), nyt_nodes.column_types()):
    if c not in aca_cols:
        aca_nodes[c] = gl.SArray.from_const(None, aca_nodes.num_rows()).astype(t)
print aca_nodes
        
nyt_cols = nyt_nodes.column_names()
for c, t in zip(aca_nodes.column_names(), aca_nodes.column_types()):
    if c not in nyt_cols:
        nyt_nodes[c] = gl.SArray.from_const(None, nyt_nodes.num_rows()).astype(t)
print nyt_nodes

+----------------------------+-------------------------------+
|             id             |             author            |
+----------------------------+-------------------------------+
|   10.1001/jama.2013.5892   | Patty J. Nelemans, Rob A. ... |
|    10.1056/nejmsa053935    | Earl S. Ford, Julia A. Cri... |
| 10.1016/j.tics.2004.11.007 | Susan E. Rogers, Daniel J.... |
|     10.1002/oby.20384      | Debra L. Roter, Kimberly A... |
|    10.1056/NEJMp1215606    | Sean Palfrey, Judith S. Pa... |
|    10.1056/nejmsa042657    | Christopher P. Neukermans,... |
|   10.1161/01.cir.67.1.24   | T. D. Noakes, L. H. Opie, ... |
|  10.1177/0363546511433279  | J. E. Shealy, S. Kim, C. F... |
|   10.3357/asem.2538.2009   | Sebastien Villard, Yawen Y... |
|   10.1001/jama.2010.303    |      Brenda R. Hemmelgarn     |
+----------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|        publication_name       |           publishe

In [7]:
# Append the two SFrames
nodes = nyt_nodes.append(aca_nodes)
nodes

id,author,title,news_desk,section_name,pub_date,topic_id
0,JAN HOFFMAN,'Fat Talk' Compels butCarries a Cost ...,None,Health,2013-05-27 13:15:43+00:00,2
1,ANAHAD O'CONNOR,Really? The Claim: FreshProduce Has More ...,None,Health,2013-05-27 14:00:36+00:00,3
2,RONI CARYN RABIN,The New Rules forSunscreen ...,None,Health,2013-05-27 15:30:46+00:00,1
3,JAN HOFFMAN,Understanding Hoarding,National,Health,2013-05-28 00:00:00+00:00,1
4,DONALD G. McNEIL Jr,New Tools to Hunt NewViruses ...,Science,Health,2013-05-28 00:00:00+00:00,0
5,DONALD G. McNEIL Jr,Polio Reappears in theHorn of Africa ...,Science,Health,2013-05-28 00:00:00+00:00,1
6,BRIAN FIDELMAN,The Roving Runner Rides aBike ...,None,Health,2013-05-28 17:01:33+00:00,0
7,GRETCHEN REYNOLDS,Is It Better to Walk orRun? ...,None,Health,2013-05-29 00:01:50+00:00,0
8,JUDITH GRAHAM,What Not to Say at theBedside ...,None,Health,2013-05-29 13:05:04+00:00,1
9,ANAHAD O'CONNOR,Winning the MRSA Battlein Hospitals ...,None,Health,2013-05-29 17:47:59+00:00,4


In [8]:
nodes.save('article_network/nodes.csv')

In [9]:
# Make the graph
# For our purposes we'll only use the academic nodes for which we have eigenfactor
filtered_aca = aca_nodes.dropna('eigenfactor')
nodes = nyt_nodes.append(filtered_aca)
edges = edges.filter_by(filtered_aca['id'], 'target')
g = gl.SGraph(vertices=nodes, edges=edges, vid_field='id', src_field='source', dst_field='target')
g

SGraph({'num_edges': 1744, 'num_vertices': 3001})
Vertex Fields:['__id', 'author', 'title', 'news_desk', 'section_name', 'pub_date', 'topic_id', 'node_type', 'publication_name', 'publisher', 'eigenfactor', 'open_access']
Edge Fields:['__src_id', '__dst_id']

In [10]:
m = gl.degree_counting.create(g)
verts = g.vertices.join(m.get('graph').vertices[['__id' , 'out_degree']], on='__id', how='left')
nyt_verts = verts.filter_by(['nyt'], 'node_type')

# remove isolates
nyt_verts = nyt_verts[nyt_verts['out_degree'] > 0]
nyt_verts

__id,author,title,news_desk,section_name,pub_date,topic_id
1902,NICHOLAS BAKALAR,High Body Mass Linked toHearing Loss in Women ...,None,Health,2013-12-05 16:32:32+00:00,3
1883,ANAHAD O'CONNOR,Really? Yawning IsContagious Between Dogs ...,None,Health,2013-08-19 10:28:21+00:00,2
1879,NICHOLAS BAKALAR,Reasons to Get a Flu Shot,None,Health,2013-06-21 11:42:12+00:00,4
1875,PAULA SPAN,A Search for Harmony,None,Health,2013-06-20 12:19:17+00:00,1
1633,DONALD G. McNEIL Jr,Understanding theProtective Side of De ...,Science,Health,2013-07-09 00:00:00+00:00,3
522,ANAHAD O'CONNOR,Can You Read the Face ofVictory? ...,None,Health,2013-01-23 13:56:38+00:00,3
86,ANAHAD O'CONNOR,Sucking Your Child'sPacifier Clean May Have ...,None,Health,2013-05-06 00:01:48+00:00,4
243,DONALD G. McNEIL Jr,Possible Path to Vaccinefor AIDS Is Suggested ...,National,Health,2013-04-04 00:00:00+00:00,0
1091,GRETCHEN REYNOLDS,This Is Your Brain onCoffee ...,None,Health,2013-06-06 00:01:32+00:00,0
486,JANE E. BRODY,A Check on Physicals,None,Health,2013-01-21 00:01:06+00:00,3


In [11]:
eigen_edges = edges.join(aca_nodes[['id', 'eigenfactor']], on={'target':'id'}, how='left')

# We need to handle the zero values before doing the log transformation
# min_val = eigen_edges[eigen_edges['eigenfactor'] > 0]['eigenfactor'].min()
# eigen_edges['eigenfactor'] = eigen_edges['eigenfactor'].apply(lambda x: nplog(x) if x != 0.0 else nplog(min_val))

avg_eigen = eigen_edges.groupby('source', {'avg_eigenfactor': agg.MEAN('eigenfactor')})
nyt_verts = nyt_verts.join(avg_eigen, on={'__id':'source'}, how='left')

In [12]:
print len(nyt_verts)
print len(nyt_verts['avg_eigenfactor'].dropna())

498
498


In [13]:
print pearsonr(nyt_verts['out_degree'], nyt_verts['avg_eigenfactor'])
print spearmanr(nyt_verts['out_degree'], nyt_verts['avg_eigenfactor'])

(0.18513169268467122, 3.2247006073188479e-05)
SpearmanrResult(correlation=0.29584752297966482, pvalue=1.617946829061244e-11)


In [14]:
nyt_verts.save('csvs/nyt_node_stats.csv')

In [15]:
no_outlier = nyt_verts.filter_by(['1013'], '__id', exclude=True)
print pearsonr(no_outlier['out_degree'], no_outlier['avg_eigenfactor'])
print spearmanr(no_outlier['out_degree'], no_outlier['avg_eigenfactor'])

(0.3056863684535821, 3.2824311451225699e-12)
SpearmanrResult(correlation=0.29977091196210898, pvalue=8.8748442760305829e-12)


In [16]:
by_author = nyt_verts.groupby('author', {'avg_eigenfactor':agg.MEAN('avg_eigenfactor'), 'citations_used':agg.MEAN('out_degree')})

In [17]:
by_author.save('csvs/nyt_ciation_stats_by_author.csv')

In [18]:
print pearsonr(by_author['citations_used'], by_author['avg_eigenfactor'])
print spearmanr(by_author['citations_used'], by_author['avg_eigenfactor'])

(0.094020233838359196, 0.52960547543795577)
SpearmanrResult(correlation=0.5045854409017394, pvalue=0.00029839692421797177)


In [19]:
aca_date_edges = edges.join(aca_nodes[['id', 'pub_date']], on={'target':'id'}, how='left')
citation_spans = aca_date_edges.groupby('source', {'earliest_citation':agg.MIN('pub_date'), 'latest_citation':agg.MAX('pub_date')})
citation_spans['citation_span'] = citation_spans.apply(lambda x: (x['latest_citation'] - x['earliest_citation']).total_seconds())
citation_spans

source,latest_citation,earliest_citation,citation_span
1373,2012-02-01 00:00:00,2011-03-10 00:00:00,28339200.0
345,2012-04-01 00:00:00,2012-04-01 00:00:00,0.0
1605,2013-01-01 00:00:00,2012-08-27 00:00:00,10972800.0
1563,2012-08-01 00:00:00,2012-02-01 00:00:00,15724800.0
87,2013-04-30 00:00:00,2013-04-30 00:00:00,0.0
787,2013-07-17 00:00:00,2007-08-15 00:00:00,186883200.0
615,2013-04-02 00:00:00,1971-01-01 00:00:00,1333324800.0
1294,2012-11-22 00:00:00,2012-11-22 00:00:00,0.0
1583,2012-08-01 00:00:00,2012-02-01 00:00:00,15724800.0
1910,2012-09-01 00:00:00,2012-09-01 00:00:00,0.0


In [20]:
nyt_verts = nyt_verts.join(citation_spans, on={'__id':'source'}, how='left')

In [21]:
# Convert the earliest and latest times to strings for Tableau
tmp = nyt_verts
tmp['earliest_citation'] = tmp['earliest_citation'].datetime_to_str(str_format='%Y-%m-%d')
tmp['latest_citation'] = tmp['latest_citation'].datetime_to_str(str_format='%Y-%m-%d')
tmp.save('csvs/nyt_node_stats.csv')

tmp

dtype: str
Rows: 498
['2013-12-01', '2013-08-07', '2013-06-19', '2013-04-11', '2012-05-29', '2012-11-29', '2010-06-21', '2013-04-03', '2012-12-09', '2012-11-20', '2014-01-01', '2014-01-01', '2012-02-15', '2013-03-20', '2013-03-01', '2011-06-24', '2013-02-20', '2010-09-01', '2013-08-01', '2010-03-01', '2013-01-29', '2013-02-06', '2013-02-27', '2013-06-18', '2012-12-01', '2012-03-01', '2013-05-01', '2013-04-11', '2011-07-01', '2013-09-25', '2011-04-01', '2013-06-13', '2013-06-06', '2014-04-09', '2013-04-10', '2013-05-29', '2013-06-26', '2009-02-01', '2013-10-01', '2013-04-25', '2013-09-04', '2013-01-29', '2013-01-30', '2013-09-05', '2011-02-01', '2015-01-01', '2013-10-16', '2013-12-01', '2012-06-01', '2013-12-11', '2009-11-01', '2013-08-08', '2013-04-08', '2010-05-27', '2010-11-30', '2013-01-01', '2012-01-19', '2014-01-01', '2011-06-13', '2013-04-04', '2013-01-28', '2013-12-23', '2016-01-01', '2013-01-01', '2012-08-01', '2009-02-27', '2013-01-01', '2013-05-01', '2012-09-12', '2013-07-01'